In [34]:
from pathlib import Path
import pandas as pd
from utils import *
import json
from json import JSONDecodeError
from mutagen.mp3 import MP3
import pandas as pd
import string
import re
import shutil

DATA_FOLDER = Path('../audio_database/2022_1_4_/processed/')
SAMU_FOLDER = Path('../audio_database/samu_files/')

In [22]:
for file in DATA_FOLDER.rglob('*.txt'):
    with open(file, "r", encoding="utf-8") as transcript_file:
        transcript = transcript_file.read()
        transcript_lower = transcript.lower()

        for prefix in ['92']:  # Loop over both prefixes
            index = 0
            while index != -1:
                index = transcript_lower.find(prefix, index)
                if index != -1:
                    start = max(0, index - 100)
                    end = min(len(transcript), index + 100)
                    snippet = transcript[start:end]
                    print(snippet)
                    index += 2  # Move past the current prefix to search for the next one


 naissance ?  Le 15 mai 1982.  Vous êtes dans quelle ville, quel arrondissement ?  Anthony, dans le 92.  A quelle adresse ?  7 rue de Champagne.  Malheureusement, je n'ai pas de médecin qui circule au
et leur demander les coordonnées du médecin de garde pour votre secteur  ou appeler directement SOS 92.  D'accord.  D'accord.  Au revoir, madame.  Au revoir.
x s'impliquer la chose.  C'est le bâtiment A au deuxième étage.  Et le code de l'ascenseur, c'est 1992 A.  C'est le bâtiment A.  Et sur le palais au deuxième, c'est droite, gauche, milieu ?  Alors, lo
 ?  Le 13 septembre 2005.  Vous êtes dans quelle ville ? Quel arrondissement ?  Vina de la Garenne, 92 390.  À quelle adresse ?  5 Allée, 8 Jouvet.  D'accord. Qu'est-ce qu'elle a comme symptôme ?  En 
ci, S-E-R-A-C-I.  Madame Ferracci, c'était pour Paris 16e ?  Alors non, mais à Saint-Cloud, dans le 92.  Saint-Cloud. Votre numéro de téléphone, c'est le 06-86-03-66, c'est ça ?  C'est exactement ça. 
 appeliez de nouveau le 15 et que vous

In [17]:
transcripts_with_SAMU = set()

for file in DATA_FOLDER.rglob('*.txt'):
    with open(file, "r", encoding="utf-8") as transcript_file:
        transcript = transcript_file.read().lower()

        if '92' in transcript:
            transcripts_with_SAMU.add(file)
            print(transcript)

# Copy folders
for file_path in transcripts_with_SAMU:
    dest_path = SAMU_FOLDER / file_path.relative_to(DATA_FOLDER).parent
    dest_path.mkdir(parents=True, exist_ok=True)
    shutil.copytree(file_path.parent, dest_path, dirs_exist_ok=True)


 sos médecin, bonjour.  oui, bonjour.  je me permets de vous contacter pour voir s'il y a un médecin qui pourrait se déplacer à domicile.  c'est pour vous, médecin ?  oui. je vous rassure, j'ai fait le test du covid hier, ce n'est pas le covid.  alors, quel est votre numéro de téléphone, s'il vous plaît ?  alors, attendez, je vais vous donner, parce que c'est un nouveau, donc je ne connais pas par cœur.  c'est le 076133-6145.  votre nom de famille ?  bonzon, b-o-n-z-o-n.  et votre prénom ?  stéphanie.  votre date de naissance ?  le 15 mai 1982.  vous êtes dans quelle ville, quel arrondissement ?  anthony, dans le 92.  a quelle adresse ?  7 rue de champagne.  malheureusement, je n'ai pas de médecin qui circule aujourd'hui à anthony.  quels sont vos symptômes en dehors de la toux ?  la toux, fièvre, courbe acturée partout.  d'accord.  alors, ce que vous pouvez faire, madame, c'est appeler le 15 et leur demander les coordonnées du médecin de garde pour votre secteur  ou appeler directemen

In [70]:
def extract_year(text):
    year_pattern = r'\b(19\d{2}|20\d{2})\b'
    return re.findall(year_pattern, text)

def extract_numerical_full_date(text):
    full_date_pattern = r'\b(0?[1-9]|[12][0-9]|3[01])[-/](0?[1-9]|1[0-2])[-/](\d{2}|\d{4})\b'
    return re.findall(full_date_pattern, text)

def extract_full_date_with_month(text):
    month_names = '|'.join(['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre'])
    date_with_month_pattern = r'\b(0?[1-9]|[12][0-9]|3[01])\s+(' + month_names + r')\s+(\d{2}|\d{4})\b'
    return re.findall(date_with_month_pattern, text)

def extract_phone_numbers(transcript):
    phone_regex = r'(06|07)(\d{8})'

    transcript = transcript.replace(" ", "")
    transcript = transcript.translate(str.maketrans('', '', string.punctuation))
    match = re.search(phone_regex, transcript)
    match = match.group() if match else match
    return match

def extract_postcodes(transcript):
    # Existing pattern for postcodes like '75015' or '75-015'
    postcode_pattern = r'\b(75|77|78|91|92|93|94|95)(?:-?)(\d{3})\b'
    raw_postcodes = re.findall(postcode_pattern, transcript)

    # Additional pattern for postcodes written as '19ème'
    arrondissement_pattern = r'\b(\d{1,2})[eè]me\b'
    arrondissements = re.findall(arrondissement_pattern, transcript)

    # Formatting arrondissements to Paris postcodes (e.g., '75019' for '19ème')
    formatted_arrondissements = ['75' + arr.zfill(3) for arr in arrondissements]

    # Combining and returning all found postcodes
    return [''.join(match) for match in raw_postcodes] + formatted_arrondissements

def extract_personal_info(data_folder):
    personal_info = {}

    address_pattern = r'\b(\d+\s+(rue|boulevard|avenue)\s+[a-zA-Zéèàêûôâîç\s-]+)\b'
    number_sequence_pattern = r'\b\d+\b'

    for file_path in data_folder.rglob('*.txt'):
        with open(file_path, 'r', encoding='utf-8') as file:
            transcript = file.read()

            personal_info[str(file_path)] = {
                'date_of_birth': {
                    'year_only': extract_year(transcript),
                    'numerical_full_date': extract_numerical_full_date(transcript),
                    'full_date_with_month': extract_full_date_with_month(transcript)
                },
                'addresses': re.findall(address_pattern, transcript),
                'postcodes': extract_postcodes(transcript),
                'phone_numbers': extract_phone_numbers(transcript),
                'other_findings': re.findall(number_sequence_pattern, transcript),
            }

    return personal_info

info = extract_personal_info(DATA_FOLDER)
info

{'..\\audio_database\\2022_1_4_\\processed\\2022_1_4_0_0_24_ch25\\2022_1_4_0_0_24_ch25_transcription.txt': {'date_of_birth': {'year_only': [],
   'numerical_full_date': [],
   'full_date_with_month': []},
  'addresses': [],
  'postcodes': ['75013'],
  'phone_numbers': None,
  'other_findings': ['1577']},
 '..\\audio_database\\2022_1_4_\\processed\\2022_1_4_0_10_18_ch17\\2022_1_4_0_10_18_ch17_transcription.txt': {'date_of_birth': {'year_only': [],
   'numerical_full_date': [],
   'full_date_with_month': []},
  'addresses': [],
  'postcodes': ['75014', '75013', '75014', '75005', '75005', '75005'],
  'phone_numbers': '0658671881',
  'other_findings': ['16',
   '39',
   '14',
   '0',
   '06',
   '58',
   '67',
   '1881',
   '2',
   '2',
   '16',
   '16',
   '6',
   '7',
   '2',
   '16',
   '39']},
 '..\\audio_database\\2022_1_4_\\processed\\2022_1_4_0_11_42_ch25\\2022_1_4_0_11_42_ch25_transcription.txt': {'date_of_birth': {'year_only': ['2000'],
   'numerical_full_date': [],
   'full_date_